In [13]:
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import netCDF4 as nc
import match
from netCDF4 import Dataset
import xarray as xr
import os 
import pandas as pd  
from datetime import datetime

# 生成文件目录

In [3]:
start_year = 2011
end_year = 2021

folder_path = 'D:/data/3DTS/2011-2021/'  # 文件夹路径
file_names = os.listdir(folder_path)  
len(file_names)  

132

In [14]:
file_new = [] 
for i in range(len(file_names)):
    three_dt = xr.open_dataset(folder_path+file_names[i])
    file_new.append(three_dt)

In [15]:
len(file_new)

132

In [16]:
data = xr.concat(file_new,dim='time')
data_path = r'D:/data/3DTS/2011-2021_concat.nc' 
data.to_netcdf(data_path)

# 读取合并数据

In [17]:
nc = Dataset(data_path)
print(nc.variables.keys())

for var in nc.variables.keys():
    data = nc.variables[var][:].data
    print(var,data.shape)

dict_keys(['time', 'depth', 'latitude', 'longitude', 'so', 'to', 'ugo', 'vgo', 'zo', 'mlotst'])
time (132,)
depth (50,)
latitude (689,)
longitude (1440,)
so (132, 50, 689, 1440)
to (132, 50, 689, 1440)
ugo (132, 50, 689, 1440)
vgo (132, 50, 689, 1440)
zo (132, 50, 689, 1440)
mlotst (132, 689, 1440)


In [19]:
nc.variables['to'][:].mask.shape 

(132, 50, 689, 1440)

# 记录数据

In [23]:
lat = []
lon = []
to = []
so = []
time = []
depth = [] 

In [24]:
to = nc.variables['to'][:].data
so = nc.variables['so'][:].data

to_mask = nc.variables['to'][:].mask
so_mask = nc.variables['so'][:].mask

depth = nc.variables['depth'][:].data

In [38]:
to[to == 32767] = np.nan
so[so == 32767] = np.nan

## 获取 lat lon time 

In [25]:
temp = Dataset(folder_path + file_names[0])
lat.append(temp.variables['latitude'][:].data)
lon.append(temp.variables['longitude'][:].data)

In [26]:
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [27]:
len(time)

132

# 保存数据

In [29]:
len(depth)

50

In [34]:
time[0]

0

In [39]:
# 只保存风速和风速异常
#创建新文件
#new_NC.close()
new_NC = Dataset("D:/data/3DTS/2011-2021_3dt_correct.nc", 'w', format='NETCDF4')

new_NC.createDimension('time', len(time))
new_NC.createDimension('depth', len(depth))
new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))

#定义变量，这里需要规定变量的类型，以及限制它的维度
#可以看到，四个与数据相关的变量，其由另外三个基本维度约束

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2011-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T' 


new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f',("depth"))

new_NC.createVariable('to', 'f', ("time","depth","lat","lon"))
new_NC.createVariable('so', 'f', ("time","depth","lat","lon"))
new_NC.createVariable('to_mask', 'f', ("time","depth","lat","lon"))
new_NC.createVariable('so_mask', 'f', ("time","depth","lat","lon"))



#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['depth'][:] = np.array(depth)
new_NC.variables['time'][:] = np.array(time)

new_NC.variables['to'][:]=np.array(to)
new_NC.variables['to_mask'][:]=np.array(to_mask)
new_NC.variables['so'][:]=np.array(so)
new_NC.variables['so_mask'][:]=np.array(so_mask)


#最后记得关闭文件
new_NC.close()

